# 知識點
- Transfer Learning 的概念是利用過去訓練過的結構與學習到的權重來加快這次的訓練任務，這是因為不同的分類任務間仍有許多共享的特徵。
- 通常淺層的卷積核學到比較粗略、泛用的特徵，因此在做 Transfer Learning 時，當新的 Dataset 與原本的 Dataset 相近時，可以考慮不更新淺層 Kernel 的權重(freeze)，又或是資料不足，擔心 Overfitting 時也可以使用。
- Freeze 的另一個好處是能加快訓練，然而當訓練資料的特徵差異太大時，還是建議可以全部開啟或只 freeze 少數層，而要 freeze 具體的層數，並沒有一定的規範。

## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras

In [2]:
input_tensor = keras.layers.Input(shape=(32, 32, 3))

'''Xception 架構'''
base_model = keras.applications.xception.Xception(weights='imagenet',
                                                  include_top=False)
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 添加層數

In [3]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(128, activation='relu')(avg)
x = keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)
print("Model 深度：", len(model.layers))

Model 深度： 136


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print(x_train.shape)

(50000, 32, 32, 3)


In [6]:
# Normalize
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    return X_train, X_test

x_train, x_test = normalize(x_train, x_test)

# OneHot
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

## Training

In [7]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 102s 2ms/sample - loss: 1.4506 - accuracy: 0.4875
Epoch 2/100
50000/50000 [==============================] - 42s 837us/sample - loss: 1.0482 - accuracy: 0.6448
Epoch 3/100
50000/50000 [==============================] - 43s 854us/sample - loss: 0.9246 - accuracy: 0.6866
Epoch 4/100
50000/50000 [==============================] - 42s 842us/sample - loss: 0.8379 - accuracy: 0.7139
Epoch 5/100
50000/50000 [==============================] - 42s 834us/sample - loss: 0.7600 - accuracy: 0.7384
Epoch 6/100
50000/50000 [==============================] - 42s 844us/sample - loss: 0.6938 - accuracy: 0.7624
Epoch 7/100
50000/50000 [==============================] - 41s 823us/sample - loss: 0.6257 - accuracy: 0.7842
Epoch 8/100
50000/50000 [==============================] - 42s 838us/sample - loss: 0.5674 - accuracy: 0.8054
Epoch 9/100
50000/50000 [==============================] - 42s 835us/sample - loss: 0.5115 - accur

KeyboardInterrupt: 